In [11]:
import pandas as pd
import re
from fuzzywuzzy import process

# Function to clean up product name by removing everything after '(' including the bracket and its content
def clean_product_name(product_name):
    # Regular expression to remove everything from the first '(' onward, including '('
    cleaned_name = re.sub(r"\(.*", "", product_name).strip()
    return cleaned_name

# Load the CSV files into pandas DataFrames
df1 = pd.read_csv("HaComp_CPU_List.csv")  # Replace with your actual file paths
df2 = pd.read_csv("AnPhat_CPU.csv")
df3 = pd.read_csv("NguyenCong_CPU_list.csv")


# Load the CPU performance file
cpu_perf_df = pd.read_csv("CPU_Performance_list.csv")

# Remove the 'Name' column from df2 (the one with 4 columns)
df2 = df2.drop(columns=['Name'], errors='ignore')  # 'errors=ignore' to avoid error if column doesn't exist

# Clean the 'Product Name' column in each DataFrame
df1['Product Name'] = df1['Product Name'].apply(clean_product_name)
df2['Product Name'] = df2['Product Name'].apply(clean_product_name)
df3['Product Name'] = df3['Product Name'].apply(clean_product_name)
cpu_perf_df['Product Name'] = cpu_perf_df['Product Name'].apply(clean_product_name)

# Rename columns for clarity with e-shop names
df1 = df1.rename(columns={'Original Price': 'Original Price HaComp', 'Discounted Price': 'Discounted Price HaComp'})
df2 = df2.rename(columns={'Original Price': 'Original Price AnPhat', 'Discounted Price': 'Discounted Price AnPhat'})
df3 = df3.rename(columns={'Original Price': 'Original Price NguyenCongPC', 'Discounted Price': 'Discounted Price NguyenCongPC'})

# Merge the DataFrames on 'Product Name' using an outer join
merged_df = pd.merge(df1, df2, on='Product Name', how='outer')
merged_df = pd.merge(merged_df, df3, on='Product Name', how='outer')

# Merge the CPU performance scores with the merged DataFrame on 'Product Name'
merged_df = pd.merge(merged_df, cpu_perf_df[['Product Name', 'Score', 'Normalized Score (%)']], on='Product Name', how='left')

# Rename 'Normalized Score' column to 'Normalized Score (%)'
merged_df = merged_df.rename(columns={'Normalized Score': 'Normalized Score (%)'})

# Optionally, handle missing values (NaNs) if needed, e.g., fill with 0 or other placeholder
# merged_df = merged_df.fillna(0)

# Save the merged DataFrame to a new CSV file
merged_df.to_csv("merged_data_with_performance.csv", index=False)

# Print a sample of the merged DataFrame
print(merged_df.head())

                        Product Name Original Price HaComp  \
0  BỘ VI XỬ LÝ AMD ATHLON 3000G TRAY                   NaN   
1               CPU AMD ATHLON 3000G            1.699.000₫   
2              CPU AMD RYZEN 3 3200G            3.299.000₫   
3               CPU AMD RYZEN 3 4100            2.499.000₫   
4       CPU AMD RYZEN 3 4100 3.8 GHZ                   NaN   

  Discounted Price HaComp Original Price AnPhat Discounted Price AnPhat  \
0                     NaN                   NaN                     NaN   
1              1.219.000₫                   NaN                     NaN   
2              1.899.000₫                   NaN                     NaN   
3              1.779.000₫                   NaN                     NaN   
4                     NaN             2.499.000               1.669.000   

  Original Price NguyenCongPC Discounted Price NguyenCongPC  Score  \
0                  1.790.000Đ                      800.000Đ    NaN   
1                         NaN       